In [1]:
from PIL import Image
import numpy as np

In [2]:
def getblocks(image: np.ndarray, blockshape: tuple, moveAxis: bool = True, info: bool = False, addChannel: bool = True) -> np.ndarray:
    '''
    takes the array of image in grey= 2D and in RGB = 3D
    takes the numpy array and converts it the the blocks in the fastest way
    '''
    if(info):
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)

    oldshape = list(image.shape)
    if addChannel and len(image.shape) == 2:
        mode = "grey"
        image = image.reshape((*image.shape, 1))
    else:
        mode = "color"

    if addChannel:
        img_height, img_width, channels = image.shape
    else:
        img_height, img_width = image.shape

    tile_height, tile_width = blockshape

    if addChannel:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels
    else:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width

    def printinfo():
        print("Old Shape:", oldshape)
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)
        print("New Shape Initial:", shp)
        print("img_height % tile_height != 0 :", img_height % tile_height != 0)
        print("img_width % tile_width != 0 :", img_width % tile_width != 0)

    if img_height % tile_height != 0 or img_width % tile_width != 0:
        print("warning: Block size is not fit for the image!")
        printinfo()

    if(info):
        printinfo()

    tiled_array = image.reshape(shp)
    tiled_array = tiled_array.swapaxes(1, 2)

    if moveAxis:
        if(addChannel):
            tiled_array = tiled_array.reshape(-1,
                                              *(tile_height, tile_width, channels))
            tiled_array = np.moveaxis(tiled_array, source=len(
                tiled_array.shape)-1, destination=1)
        else:
            tiled_array = tiled_array.reshape(-1, *(tile_height, tile_width))

    return tiled_array


In [3]:
def combineBlocks(tiled_array: np.ndarray, imageshape: tuple, blockshape: tuple, movedAxis: bool = True, channel: bool = True) -> np.ndarray:

    if channel:
        if len(imageshape) == 2:
            mode = "grey"
            imageshape = *imageshape, 1
        else:
            mode = "color"

    if channel:
        img_height, img_width, channels = imageshape
    else:
        img_height, img_width = imageshape

    tile_height, tile_width = blockshape

    if movedAxis:
        image = tiled_array.copy()
        if(channel):
            image = image.reshape(img_height//tile_height, tile_height,
                                  img_width//tile_width, tile_width, channels)
            swapaxisShape = list(image.shape)
            swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
            image = image.reshape(swapaxisShape)
            image = image.swapaxes(1, 2)
        else:
            f = image.reshape(img_height//tile_height, tile_height,
                              img_width//tile_width, tile_width)
            swapaxisShape = list(f.shape)
            swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
            tmp = f.reshape(swapaxisShape)
            image = tmp.swapaxes(1, 2)
    else:
        image = tiled_array
        # IDK have completed this else case. Btw we aren't using this case lol :)

    return image.reshape(imageshape)


In [4]:
image = np.arange(1,(512*512)+1).reshape(512,512)
image

array([[     1,      2,      3, ...,    510,    511,    512],
       [   513,    514,    515, ...,   1022,   1023,   1024],
       [  1025,   1026,   1027, ...,   1534,   1535,   1536],
       ...,
       [260609, 260610, 260611, ..., 261118, 261119, 261120],
       [261121, 261122, 261123, ..., 261630, 261631, 261632],
       [261633, 261634, 261635, ..., 262142, 262143, 262144]])

In [5]:
# img=Image.open("lena_gray.gif").convert("L")
# image=np.array(img)

In [6]:
image

array([[     1,      2,      3, ...,    510,    511,    512],
       [   513,    514,    515, ...,   1022,   1023,   1024],
       [  1025,   1026,   1027, ...,   1534,   1535,   1536],
       ...,
       [260609, 260610, 260611, ..., 261118, 261119, 261120],
       [261121, 261122, 261123, ..., 261630, 261631, 261632],
       [261633, 261634, 261635, ..., 262142, 262143, 262144]])

In [7]:
image.shape,image.strides

((512, 512), (2048, 4))

In [8]:
blocksize=(16,16)

In [9]:
# image=image.reshape(*image.shape, 1)

In [10]:
f=getblocks(image, blockshape=blocksize,addChannel=False)
f.shape, f.strides, f

((1024, 16, 16),
 (1024, 64, 4),
 array([[[     1,      2,      3, ...,     14,     15,     16],
         [   513,    514,    515, ...,    526,    527,    528],
         [  1025,   1026,   1027, ...,   1038,   1039,   1040],
         ...,
         [  6657,   6658,   6659, ...,   6670,   6671,   6672],
         [  7169,   7170,   7171, ...,   7182,   7183,   7184],
         [  7681,   7682,   7683, ...,   7694,   7695,   7696]],
 
        [[    17,     18,     19, ...,     30,     31,     32],
         [   529,    530,    531, ...,    542,    543,    544],
         [  1041,   1042,   1043, ...,   1054,   1055,   1056],
         ...,
         [  6673,   6674,   6675, ...,   6686,   6687,   6688],
         [  7185,   7186,   7187, ...,   7198,   7199,   7200],
         [  7697,   7698,   7699, ...,   7710,   7711,   7712]],
 
        [[    33,     34,     35, ...,     46,     47,     48],
         [   545,    546,    547, ...,    558,    559,    560],
         [  1057,   1058,   1059, ...

In [11]:
combineBlocks(f, (512, 512), blockshape=blocksize, channel=False)


array([[     1,      2,      3, ...,    510,    511,    512],
       [   513,    514,    515, ...,   1022,   1023,   1024],
       [  1025,   1026,   1027, ...,   1534,   1535,   1536],
       ...,
       [260609, 260610, 260611, ..., 261118, 261119, 261120],
       [261121, 261122, 261123, ..., 261630, 261631, 261632],
       [261633, 261634, 261635, ..., 262142, 262143, 262144]])

In [80]:
img_height, img_width = image.shape
tile_height, tile_width = blocksize


In [81]:
haha=img_height//tile_height, tile_height, img_width//tile_width, tile_width
haha

(4, 2, 4, 2)

In [82]:
a=image.reshape(haha)
a, a.shape, a.strides

(array([[[[ 1,  2],
          [ 3,  4],
          [ 5,  6],
          [ 7,  8]],
 
         [[ 9, 10],
          [11, 12],
          [13, 14],
          [15, 16]]],
 
 
        [[[17, 18],
          [19, 20],
          [21, 22],
          [23, 24]],
 
         [[25, 26],
          [27, 28],
          [29, 30],
          [31, 32]]],
 
 
        [[[33, 34],
          [35, 36],
          [37, 38],
          [39, 40]],
 
         [[41, 42],
          [43, 44],
          [45, 46],
          [47, 48]]],
 
 
        [[[49, 50],
          [51, 52],
          [53, 54],
          [55, 56]],
 
         [[57, 58],
          [59, 60],
          [61, 62],
          [63, 64]]]]),
 (4, 2, 4, 2),
 (64, 32, 8, 4))

In [83]:
b=a.swapaxes(1,2)
b, b.shape, b.strides

(array([[[[ 1,  2],
          [ 9, 10]],
 
         [[ 3,  4],
          [11, 12]],
 
         [[ 5,  6],
          [13, 14]],
 
         [[ 7,  8],
          [15, 16]]],
 
 
        [[[17, 18],
          [25, 26]],
 
         [[19, 20],
          [27, 28]],
 
         [[21, 22],
          [29, 30]],
 
         [[23, 24],
          [31, 32]]],
 
 
        [[[33, 34],
          [41, 42]],
 
         [[35, 36],
          [43, 44]],
 
         [[37, 38],
          [45, 46]],
 
         [[39, 40],
          [47, 48]]],
 
 
        [[[49, 50],
          [57, 58]],
 
         [[51, 52],
          [59, 60]],
 
         [[53, 54],
          [61, 62]],
 
         [[55, 56],
          [63, 64]]]]),
 (4, 4, 2, 2),
 (64, 8, 32, 4))

In [84]:
c=b.reshape(-1, *blocksize)
c, c.shape, c.strides


(array([[[ 1,  2],
         [ 9, 10]],
 
        [[ 3,  4],
         [11, 12]],
 
        [[ 5,  6],
         [13, 14]],
 
        [[ 7,  8],
         [15, 16]],
 
        [[17, 18],
         [25, 26]],
 
        [[19, 20],
         [27, 28]],
 
        [[21, 22],
         [29, 30]],
 
        [[23, 24],
         [31, 32]],
 
        [[33, 34],
         [41, 42]],
 
        [[35, 36],
         [43, 44]],
 
        [[37, 38],
         [45, 46]],
 
        [[39, 40],
         [47, 48]],
 
        [[49, 50],
         [57, 58]],
 
        [[51, 52],
         [59, 60]],
 
        [[53, 54],
         [61, 62]],
 
        [[55, 56],
         [63, 64]]]),
 (16, 2, 2),
 (16, 8, 4))

reversing

In [85]:
e=c.copy()
e.shape,e.strides,e
# (4096, 2, 2, 1),
#  (16, 8, 4, 4),

((16, 2, 2),
 (16, 8, 4),
 array([[[ 1,  2],
         [ 9, 10]],
 
        [[ 3,  4],
         [11, 12]],
 
        [[ 5,  6],
         [13, 14]],
 
        [[ 7,  8],
         [15, 16]],
 
        [[17, 18],
         [25, 26]],
 
        [[19, 20],
         [27, 28]],
 
        [[21, 22],
         [29, 30]],
 
        [[23, 24],
         [31, 32]],
 
        [[33, 34],
         [41, 42]],
 
        [[35, 36],
         [43, 44]],
 
        [[37, 38],
         [45, 46]],
 
        [[39, 40],
         [47, 48]],
 
        [[49, 50],
         [57, 58]],
 
        [[51, 52],
         [59, 60]],
 
        [[53, 54],
         [61, 62]],
 
        [[55, 56],
         [63, 64]]]))

In [86]:
# # e=np.moveaxis(d, source=1, destination=len(c.shape)-1)
# e = d.copy()
# e.shape, e.strides, e


In [87]:
(e==c).all()

True

In [88]:
(e.reshape(-1)==b.reshape(-1)).all()

True

In [89]:
f=e.reshape(haha)
f.shape, f.strides,f


((4, 2, 4, 2),
 (64, 32, 8, 4),
 array([[[[ 1,  2],
          [ 9, 10],
          [ 3,  4],
          [11, 12]],
 
         [[ 5,  6],
          [13, 14],
          [ 7,  8],
          [15, 16]]],
 
 
        [[[17, 18],
          [25, 26],
          [19, 20],
          [27, 28]],
 
         [[21, 22],
          [29, 30],
          [23, 24],
          [31, 32]]],
 
 
        [[[33, 34],
          [41, 42],
          [35, 36],
          [43, 44]],
 
         [[37, 38],
          [45, 46],
          [39, 40],
          [47, 48]]],
 
 
        [[[49, 50],
          [57, 58],
          [51, 52],
          [59, 60]],
 
         [[53, 54],
          [61, 62],
          [55, 56],
          [63, 64]]]]))

In [90]:
(f.reshape(-1)==b.reshape(-1)).all()

True

In [91]:
(f.reshape(-1)==a.reshape(-1)).all()

False

In [92]:
f.shape, f.strides

((4, 2, 4, 2), (64, 32, 8, 4))

In [93]:
# h=f.swapaxes(1,2)
# h.shape, h.strides, h

In [94]:
# i=h.swapaxes(1,2)
# i.shape, i.strides, i

In [95]:
# j=i.swapaxes(1,2)
# j.shape, j.strides, j

In [96]:
swapaxisShape = list(f.shape)
swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
swapaxisShape

[4, 4, 2, 2]

In [97]:
tmp=f.reshape(swapaxisShape)
tmp.shape, tmp.strides, tmp

((4, 4, 2, 2),
 (64, 16, 8, 4),
 array([[[[ 1,  2],
          [ 9, 10]],
 
         [[ 3,  4],
          [11, 12]],
 
         [[ 5,  6],
          [13, 14]],
 
         [[ 7,  8],
          [15, 16]]],
 
 
        [[[17, 18],
          [25, 26]],
 
         [[19, 20],
          [27, 28]],
 
         [[21, 22],
          [29, 30]],
 
         [[23, 24],
          [31, 32]]],
 
 
        [[[33, 34],
          [41, 42]],
 
         [[35, 36],
          [43, 44]],
 
         [[37, 38],
          [45, 46]],
 
         [[39, 40],
          [47, 48]]],
 
 
        [[[49, 50],
          [57, 58]],
 
         [[51, 52],
          [59, 60]],
 
         [[53, 54],
          [61, 62]],
 
         [[55, 56],
          [63, 64]]]]))

In [98]:
f.strides, f.shape

((64, 32, 8, 4), (4, 2, 4, 2))

In [99]:
# tmp.copy().strides, tmp.shape

In [100]:
# tmp2=f.swapaxes(1,2)
# # tmp2.shape, tmp2.strides, tmp2

In [101]:
ret=tmp.swapaxes(1,2) ## done

In [102]:
ret

array([[[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8]],

        [[ 9, 10],
         [11, 12],
         [13, 14],
         [15, 16]]],


       [[[17, 18],
         [19, 20],
         [21, 22],
         [23, 24]],

        [[25, 26],
         [27, 28],
         [29, 30],
         [31, 32]]],


       [[[33, 34],
         [35, 36],
         [37, 38],
         [39, 40]],

        [[41, 42],
         [43, 44],
         [45, 46],
         [47, 48]]],


       [[[49, 50],
         [51, 52],
         [53, 54],
         [55, 56]],

        [[57, 58],
         [59, 60],
         [61, 62],
         [63, 64]]]])

In [103]:
(ret==a).all()

True

In [104]:
ret.shape, ret.strides

((4, 2, 4, 2), (64, 8, 16, 4))

In [105]:
ret=ret.reshape((512,512))
ret

ValueError: cannot reshape array of size 64 into shape (512,512)

In [ ]:
g=f.swapaxes(1,2)
g.shape, g.strides, g

In [ ]:
(g.reshape(-1)==b.reshape(-1)).all()

In [ ]:
image.reshape(-1).shape, image.reshape(-1).strides

In [ ]:
g.reshape(-1).shape, g.reshape(-1).strides

In [ ]:
h=g.reshape(-1).reshape(haha)
h, h.shape, h.strides


In [ ]:
(h.reshape(-1)==image.reshape(-1)).all()

In [ ]:
np.set_printoptions(threshold=np.inf)
g.reshape(-1)==image.reshape(-1)

In [ ]:
h=b.swapaxes(1,2)
h.reshape(-1)